In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import json
from pandas import read_csv
from datetime import datetime

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#Just follow the link and login to your google account and get the authorization code and paste it where required.

**Importing the library package, loading the dataset and reading the train_data **

##Exploratory data analysis

In [ ]:
# load data
train_data = read_csv('/content/drive/MyDrive/PHD/train_data.csv',header = 0)
train_data['year'] = (train_data['year']-2014)# to reduce the value in order to start preparing data for model
train_data['month'] = (train_data['month'])
train_data['day'] = (train_data['day'])
train_data['city'] = train_data['city']

# load dataset
train_values = train_data.values
# specify columns to plot
groups = [0, 1, 2, 3, 4, 5]
i = 1
# plot each column
plt.figure()
for group in groups:
	plt.subplot(len(groups), 1, i)
	plt.plot(train_values[:, group])
	plt.title(train_data.columns[group], y=0.5, loc='right')
	i += 1
plt.show()

corr_coefs = train_data.corr()
print("--------Wholesome Correlation matrix---------")
print(corr_coefs)

corr_coefs1 = train_data[train_data['year']==1].corr()
print("--------2015 Correlation matrix---------")
print(corr_coefs1)

corr_coefs2 = train_data[train_data['year']==2].corr()
print("--------2016 Correlation matrix---------")
print(corr_coefs2)

corr_coefs3 = train_data[train_data['year']==3].corr()
print("--------2017 Correlation matrix---------")
print(corr_coefs3)

corr_coefs4 = train_data[train_data['year']==4].corr()
print("--------2018 Correlation matrix---------")
print(corr_coefs4)

  **To avoid warning messages**

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
def reduce_mem_usage(train_data):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in train_data.columns:
        col_type = train_data[col].dtype
        
        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return train_data

In [ ]:
pd.plotting.scatter_matrix(train_data, alpha=0.2)

**Next step would be to explore how the sales changed with day, month, year and city.**

#Data Visualization and Insights

In [ ]:
#Plotting how sales changed with year,month, day.
train_data.plot(x = 'year',y = 'sales')
plt.figure()
train_data.plot(x = 'month',y = 'sales')
plt.figure()
train_data.plot(x = 'day',y = 'sales')
plt.figure()
train_data.plot(x='medicine', y = 'sales')
plt.figure()
train_data.plot(x = 'city',y = 'sales')
plt.figure()
with open('/content/drive/MyDrive/PHD/city_dict.json') as f:
    city_data = json.load(f)
print(city_data)

Delhi (code 5) was found to be the highest selling place among all the city data present. Least selling was from other cities. Second least was Kolkata, which was a bit surprising. Because in terms of population there is not as much population difference between Kolkata and Delhi as that of Kolkata and Ahmedabad. However in the plot ahmedabad showed much higher sales than Kolkata and other cities.

The correlation matrix shows that sles did not follow same pattern of change every year. From 2015-2017 amount of medicines in the store showed highest magnitude of correlation. however in 2018 city affected more than other factors, where amount of medicines in the store affected the least. However we don't have the data for approx. 5 months.

  From previous comparison it can be said that the data doesn't follow linear relation. Hence ML model which is used later doesn't perform well in terms of RMSE. However when the data is trained using only the data from 2017-2018 only, the performance was found to be better. This suggests that concentrating on 6-8 months of data would be more useful i prediction rather than the data from 3 years.

In [ ]:
plt.hist2d(train_data['year'], train_data['sales'], bins=(50, 50), vmax=2000)
plt.colorbar()
plt.xlabel('year')
plt.ylabel('sales')

plt.figure()
plt.hist2d(train_data['month'], train_data['sales'], bins=(50, 50), vmax=2000)
plt.colorbar()
plt.xlabel('month')
plt.ylabel('sales')

plt.figure()
plt.hist2d(train_data['day'], train_data['sales'], bins=(50, 50), vmax=2000)
plt.colorbar()
plt.xlabel('day')
plt.ylabel('sales')

plt.figure()
plt.hist2d(train_data['medicine'], train_data['sales'], bins=(50, 50), vmax=2000)
plt.colorbar()
plt.xlabel('medicine')
plt.ylabel('sales')

plt.figure()
plt.hist2d(train_data['city'], train_data['sales'], bins=(50, 50), vmax=2000)
plt.colorbar()
plt.xlabel('city')
plt.ylabel('sales')

**The time-frequency plot between each feature and sales would show the frequency at which they affect the sales. As expected for year as the data is for 2015, 2016, 2017 and 2018, hence at those instances the color is warmer than anywhere else in the plot. Similarly for months, as months are from 1-12, the color is warmer in those regions. The time-frequency plot between medicine and sales shows that storage of medicine less than approximately 2500 doesn't affect the sales to some extent when compared visually.**

In [ ]:
sns.pairplot(train_data)#This scatter plot matrix shows the relationship between different parameters.

**Read discount_features csv file,concatenate discount_data with train_data. Discount should be added in the training dataset to be more effective in changing strategy for sales.**

In [ ]:
discount_data = read_csv('/content/drive/MyDrive/PHD/discount_features.csv',header = 0)
discount_data.head(5)
discount_data['date'] = pd.to_datetime(discount_data['date'])
discount_data['date']
print(discount_data.head(5))
discount_data.describe().transpose()
data_merge = pd.concat([train_data, discount_data], ignore_index=True, sort=False)

In [ ]:
from scipy.stats import ttest_ind
print(data_merge.head(5))
corr_coefs_all = train_data.corr()
print("--------Correlation matrix including discount data---------")
print(corr_coefs_all)
print(" ")

**To check format of the date since code doesn't take object datatype**

In [ ]:
footfall_data = pd.read_csv("/content/drive/MyDrive/PHD/foot_fall.csv")
footfall_data.head(5)
footfall_data=footfall_data.melt(id_vars=["city"], 
        var_name="Date", 
        value_name="Value") 
footfall_data.head(10)
city_names = pd.DataFrame.from_dict(city_data, orient="index")
city_names = city_names.reset_index()
city_names.head(5)
footfall_data['Date'] = pd.to_datetime(footfall_data['Date'])
footfall_data['Date']

**These visualizations shows that number of customers in all the years did not vary much from each other , value of customer i,e footfall**

**Data Visualization and Insights**

In [ ]:
footfall_data['year'] = footfall_data['Date'].dt.year
footfall_data['month'] = footfall_data['Date'].dt.month
footfall_data['day'] = footfall_data['Date'].dt.day
footfall_data.head(5)

import seaborn as sns
#Plotting how sales changed with year.
footfall_data.plot(x = 'year',y = 'Value')
plt.figure()
footfall_data.plot(x = 'month',y = 'Value')
plt.figure()
footfall_data.plot(x = 'day',y = 'Value')
plt.figure()
footfall_data.plot(x = 'city',y = 'Value')
plt.figure()

**Variation in different features  present in train field, hence it was necessary to further look at frequency plot. Sales plot does not show clear relationship with other parameters hence studying the frequency domain was necessary, the relationship also reflected upon correlation between parameters which is very low**

In [ ]:
plot_cols = ['year', 'month', 'day', 'city','medicine','sales']
plot_features = train_data[plot_cols]
_ = plot_features.plot(subplots=True)

plot_features = train_data[plot_cols][:480]
_ = plot_features.plot(subplots=True)

##Feature Engineering:
**(1.) Here we would look at how the frequency 
affects the sales, year or day didn't show any high peaks at year , month and day frequency which tells sales was less influenced by frequency/periodicity.
(2.)This plot shows that year frequency or day frequency did not show very clear stand-out feature which means that these two don't have so much effect on sales.**

In [ ]:
fft = tf.signal.rfft(train_data['sales'])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(train_data['sales'])
days_per_year = 365.2524
days_per_month = 365.2524/12
years_per_dataset = n_samples_h/(days_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 100000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524/12, 365.2524], labels=['1/Year', '1/month', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

**Looking at the results, it can be said that it is important to look into why there is high variation in sales among biggest cities. It would be important to look into holidays and day of the week data to explore which days are the key days to change the amount of sales** 

**Reduce Memory Footprint Pandas**

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
#load data
csv_path = '/content/drive/MyDrive/PHD/train_data.csv'

##Machine Learning model Building


1.   Data Preparation for effective model preparation.
2.   Baseline Model preparation.
3.   Find the best model and windowing method for this purpose.
4.   Use the model to predict the sales for test.csv.



In [ ]:
df = pd.read_csv(csv_path)
# slice [start:stop:step], starting from left index every right indexth record.
df = df[1::200] # THis sentence is in case you want to not take the whole data or read data over a period.

df.head()

In [ ]:
print(df.head())

df['sales'] = df['sales'].clip(upper=2500) 
# In the training set there are some outliers which may reduce the performance of the model, because outliers meaning is not usual case. 
#Hence the data is slipped above 2500 for this dataset

**Splitting Data into training, validation and test dataset at 70:20:10 ratio of the total training data.**

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

#Normalization of splitted datasets.
  **The year values are in thousands whereas month varies from 1-12 while day varies from 1-31 and medicine from zero to thousands. High variation in the dataset may cause the model to not converge. Hence Normalization was needed.
  There are many ways of normalizing the data. One is to subtract DC offset from the data and dividing standard deviation of whole training Dataset. However as year value contains only 4 unique values and in test dataset only single type of year value would be there, in that case this method can cause the year values to be Nans. 
  As Normalization meaning is to scale the data between certain limit, generally 0-1, dividing maximum would achieve the same.**

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df) / train_df.max()
val_df = (val_df) / train_df.max()
test_df = (test_df) / train_df.max()

#train_df = (train_df - train_mean) / train_std
#val_df = (val_df - train_mean) / train_std
#test_df = (test_df - train_mean) / train_std

**In time-series forecasting windowing is important to make predictions based on consecutive pair of features and labels. Different performance of model can be achieved based on the length of window and model used. Hence we are going to use different varieties of windows (wide, narrow, signle-step and multi-step).**

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Set the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
w1 = WindowGenerator(input_width=31, label_width=1, shift=1,
                     label_columns=['sales'])
w1

In [ ]:
w2 = WindowGenerator(input_width=10, label_width=1, shift=1,
                     label_columns=['sales'])
w2

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually which makes it easier to make the dataset.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:

# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])


example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'labels shape: {example_labels.shape}')

In [ ]:
w2.example = example_inputs, example_labels

In [ ]:
def plot(self, model=None, plot_col='sales', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='*', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('days')

WindowGenerator.plot = plot

In [ ]:
w2.plot()

**Built a dataset from the windows generated through make_dataset function.**

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=30)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
# Each element is an (inputs, label) pair
w2.train.element_spec

In [ ]:
MAX_EPOCHS = 2

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

#Baseline Model:

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

#Baseline model using single step window: The model made to predict immediate short length based on current data.
#Baseline model is necessary to test the performance of the base model i.e., simplest model to improve further and compare other models w.r.t the baseline model. Accordingly whichever model performs the best, would be selected to be used with the test.csv data. 
Use of linear model applies linear transform between input and output. Below implemented is one of the linear model implemented.

Evaluation of the performance of the model based on the metrics set in the model.compile was conducted above and shown below.

In [ ]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you 
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1)

wide_window = WindowGenerator(
    input_width=31, label_width=31, shift=1)

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
baseline = Baseline()
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.RootMeanSquaredError()])

#Dense model. In this model change in number of units can change the performance of the model.

In [ ]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=num_features)
])

In [ ]:
history = compile_and_fit(dense, single_step_window)

IPython.display.clear_output()

val_performance = {}
performance = {}

val_performance['Dense'] = dense.evaluate(single_step_window.val)
performance['Dense'] = dense.evaluate(single_step_window.test, verbose=2)

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each timestep is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta

In [ ]:
%%time
wide_window = WindowGenerator(
    input_width=31, label_width=31, shift=1)

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(64, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()

In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small
        # So initialize the output layer with zeros
        kernel_initializer=tf.initializers.zeros)
]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val)
performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0)
print()

In [ ]:
x = np.arange(len(performance))
width = 0.3

metric_name = 'root_mean_squared_error'
metric_index = lstm_model.metrics_names.index('root_mean_squared_error')
val_rmse = [v[metric_index] for v in val_performance.values()]
test_rmse = [v[metric_index] for v in performance.values()]

plt.bar(x - 0.17, val_rmse, width, label='Validation')
plt.bar(x + 0.17, test_rmse, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
plt.ylabel('RMSE (average over all outputs)')
_ = plt.legend()

**This shows that Residual LSTM performed better than LSTM. However the RMSE was not very different between Dense and Residual LSTM.** 

In [ ]:
for name, value in performance.items():
  print(f'{name:15s}: {value[1]:0.4f}')

In [ ]:
OUT_STEPS = 31
multi_window = WindowGenerator(input_width=31,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

##Multistep Last Baseline:

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(last_baseline)

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.RootMeanSquaredError()])

multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(repeat_baseline)

**Repeat baseline clearly shows better prediction than that of the previous two baselines.**

In [ ]:
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_linear_model)

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_dense_model)

##CNN

In [ ]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(128, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_conv_model, multi_window)

IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_conv_model)

##RNN

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(64, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

In [ ]:
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'root_mean_squared_error'
metric_index = lstm_model.metrics_names.index('root_mean_squared_error')
val_rmse = [v[metric_index] for v in multi_val_performance.values()]
test_rmse = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_rmse, width, label='Validation')
plt.bar(x + 0.17, test_rmse, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'RMSE (average over all times and outputs)')
_ = plt.legend()

In [ ]:
for name, value in multi_performance.items():
  print(f'{name:8s}: {value[1]:0.4f}')

**From the visualization and the average RMSE values, Dense and CNN are found to be most competitive ones for prediction.**

## Test using test.csv:
Load the test.csv file and as the model is trained on windows, the test.csv would also be split into windows to be used in prediction.

In [ ]:
#load data
test_csv_path = '/content/drive/MyDrive/Drug_sales/test_data.csv'

In [ ]:
df_test = pd.read_csv(test_csv_path)
IDs = df_test.pop('id')
df_test.head()

In [ ]:
final_test = df_test/df_test.max()
final_test.head()

In [ ]:
w11 = WindowGenerator(input_width=31, label_width=31, shift=1)
w11

Label column names is None, because we don't have any labels as we need to predict the sales for the test.csv dataset.

Here I have followed wide-window method for prediction in chunks followed by saving the result in a csv file.

In [ ]:
predictions_norm = []
wide_window = WindowGenerator(
    input_width=4700, label_width=4700, shift=1)
for example_inputs1, example_labels1 in wide_window.train.take(1):
  predictions_norm.append(abs(dense.predict(example_inputs1)))

predictions = np.array(predictions_norm)
n= predictions.shape[0]*predictions.shape[1]*predictions.shape[2]*predictions.shape[3]
predictions1 = predictions.reshape((n,1))
print(predictions1)
final_predictions = (predictions1*train_std['sales']) + train_mean['sales'] 
final_predictions = final_predictions[:len(IDs)]

In [ ]:
dict_ofsubmission = {'sales':final_predictions}
submission_df = pd.DataFrame(data = final_predictions,index = None, columns = ['sales'])
submission_df.to_csv('submission_sales.csv')  #It would contain more than the number of rows. In that case, just select first 0:len(df['id'])
submission_df.head()